# Business Problem Context

The company Fome Zero is a restaurant marketplace. In other words, your core
business is to facilitate the meeting and negotiations of customers and restaurants. You
restaurants register on the Fome Zero platform, which provides
information such as address, type of cuisine served, whether you have reservations, whether
deliveries and also an evaluation note of the restaurant's services and products,
among other information.

## The challenge
CEO Guerra was also recently hired and needs to better understand the business
to be able to make the best strategic decisions and further leverage the
Fome Zero, and for this, he needs an analysis of the data from the
company and that dashboards are generated, based on these analyses, to answer various questions.

# Imports

In [32]:
# Importação de Pacotes
import pandas as pd
import inflection
import numpy as np

import plotly.express as px

# Helper Variables

In [33]:
COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}

COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

# Helper Functions

In [58]:
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

def country_name(country_id):
    return COUNTRIES[country_id]

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
def color_name(color_code):
    return COLORS[color_code]

def show_dataframe_dimensions(dataframe):
    print(f"Number of Rows: {dataframe.shape[0]}")
    print(f"Number of Columns: {dataframe.shape[1]}")
    
    return None

def get_first_order_statistics(dataframe):
    # Central Tendency Metrics
    mean = pd.DataFrame(dataframe.apply(np.mean)).T
    median = pd.DataFrame(dataframe.apply(np.median)).T

    # Dispersion Metrics
    min_ = pd.DataFrame(dataframe.apply(min)).T
    max_ = pd.DataFrame(dataframe.apply(max)).T
    range_ = pd.DataFrame(dataframe.apply(lambda x: x.max() - x.min())).T
    std = pd.DataFrame(dataframe.apply(np.std)).T
    skew = pd.DataFrame(dataframe.apply(lambda x: x.skew())).T
    kurtosis = pd.DataFrame(dataframe.apply(lambda x: x.kurtosis())).T

    # Metrics Concatenation
    m = pd.concat([min_, max_, range_, mean, median, std, skew, kurtosis]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    return m

def process_data(file_path):
    df = pd.read_csv(file_path)

    df = rename_columns(df)

    df = df.drop_duplicates(subset=['restaurant_id'])

    df = df.dropna()

    # Change data types
    df['has_table_booking'] = df['has_table_booking'].astype(bool)
    df['has_online_delivery'] = df['has_online_delivery'].astype(bool)
    df['is_delivering_now'] = df['is_delivering_now'].astype(bool)

    df['country'] = df['country_code'].apply(country_name)
    df['price_type'] = df['price_range'].apply(create_price_tye)
    df['color_name'] = df['rating_color'].apply(color_name)
    df["cuisines_"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

    df = df[~df['cuisines_'].isin(['Drinks Only', 'Mineira'])]

    df = df.reset_index(drop=True)

    df.to_csv("../data/processed/data.csv", index=False)

    return df

def create_bar_plot(df, x, y, labels):
    """
    Create a bar plot using Plotly Express.
    
    Parameters:
    - df (DataFrame): The dataframe containing the data.
    - x (str): The column name for the x-axis.
    - y (str): The column name for the y-axis.
    - labels (dict): A dictionary specifying custom labels for x, y, and color (optional).
    
    Returns:
    - fig (plotly.graph_objs.Figure): The Plotly figure object.
    """
    # Create the bar plot using Plotly Express
    fig = px.bar(df, x=x, y=y,
                 height=600,  # Adjust the height as needed
                 color='country',  # Color based on mean aggregate rating
                 labels=labels,  # Custom labels
                 category_orders={x: df[x].tolist()})  # Maintain original order of categories
    
    # Adjust text position for better visibility
    fig.update_traces(textposition='outside')
    
    return fig

# Load Data

In [81]:
df = pd.read_csv('../data/raw/zomato.csv')

In [82]:
df.dtypes

Restaurant ID             int64
Restaurant Name          object
Country Code              int64
City                     object
Address                  object
Locality                 object
Locality Verbose         object
Longitude               float64
Latitude                float64
Cuisines                 object
Average Cost for two      int64
Currency                 object
Has Table booking         int64
Has Online delivery       int64
Is delivering now         int64
Switch to order menu      int64
Price range               int64
Aggregate rating        float64
Rating color             object
Rating text              object
Votes                     int64
dtype: object

In [86]:
df['Aggregate rating'].median()

4.2

# 1.0. DATA DESCRIPTION

In [18]:
df1 = df.copy()

# Imprimir as dimensões dos dados
show_dataframe_dimensions(df1)

# Show the first 5 records
df1.head()

Number of Rows: 7527
Number of Columns: 21


,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


## 1.1. Columns Description

| Column                   | Description                                                                                     |
|:-------------------------|:------------------------------------------------------------------------------------------------|
| Restaurant ID            | ID do restaurante                                                                               |
| Restaurant Name          | Nome do Restaurante                                                                             |
| Country Code             | Código do País                                                                                  |
| City                     | Nome da Cidade onde o restaurante está                                                          |
| Address                  | Endereço do restaurante                                                                         |
| Locality                 | Localização e pontos de referência do restaurante                                               |
| Locality Verbose         | Localização e pontos de referência do restaurante (Mais informações)                            |
| Longitude                | Ponto geográfico de Longitude do Restaurante                                                    |
| Latitude                 | Ponto geográfico de Latitude do Restaurante                                                     |
| Cuisines                 | Tipos de Culinária servidos no restaurante                                                      |
| Average Cost for two     | Preço Médio de um prato para duas pessoas no restaurante                                        |
| Currency                 | Moeda do país                                                                                   |
| Has Table booking        | Se o restaurante possui serviços de reserva; 1 - Sim; 0 - Não                                   |
| Has Online delivery      | Se o restaurante possui serviços de pedido on-line; 1 - Sim; 0 - Não                            |
| Is delivering now        | Se o restaurante faz entregas; 1 - Sim; 0 - Não                                                 |
| Switch to order menu     | -                                                                                               |
| Price range              | Variação de preços do restaurante; 1 a 4 - Quanto maior o valor, mais caro serão os pratos       |
| Aggregate rating         | Nota média do restaurante                                                                       |
| Rating color             | Código Hexadecimal da cor do restaurante com base em sua nota média                             |
| Rating text              | Categoria em que o restaurante está com base em sua nota média                                  |
| Votes                    | Quantidade de avaliações que o restaurante já recebeu                                           |


## 1.2 Rename Columns

In [19]:
df1 = rename_columns(df1)

In [20]:
print(df1.columns)

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')


## 1.3. Check Data Types

In [21]:
df1.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
dtype: object

## 1.4. Check NA

In [22]:
df1.isna().sum()

restaurant_id            0
restaurant_name          0
country_code             0
city                     0
address                  0
locality                 0
locality_verbose         0
longitude                0
latitude                 0
cuisines                15
average_cost_for_two     0
currency                 0
has_table_booking        0
has_online_delivery      0
is_delivering_now        0
switch_to_order_menu     0
price_range              0
aggregate_rating         0
rating_color             0
rating_text              0
votes                    0
dtype: int64

## 1.5. Checks the number of unique values

In [23]:
df1.nunique()

restaurant_id           6942
restaurant_name         5914
country_code              15
city                     125
address                 6760
locality                2272
locality_verbose        2357
longitude               6846
latitude                6833
cuisines                2832
average_cost_for_two     156
currency                  12
has_table_booking          2
has_online_delivery        2
is_delivering_now          2
switch_to_order_menu       1
price_range                4
aggregate_rating          30
rating_color               7
rating_text               28
votes                   1739
dtype: int64

## 1.6. Descriptive Statistics

In [24]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])

### 1.6.1. Numerical Data

In [25]:
get_first_order_statistics(num_attributes)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.000000,1.904028e+07,1.903973e+07,1.055689e+07,7.701457e+06,7.074671e+06,-0.087788,-1.726571
1,country_code,1.000000,2.160000e+02,2.150000e+02,9.306536e+01,3.000000e+01,9.902465e+01,0.258687,-1.849127
2,longitude,-122.700455,1.753106e+02,2.980110e+02,3.376409e+01,7.378512e+01,7.739010e+01,-0.680775,-0.596714
3,latitude,-41.330428,5.597698e+01,9.730741e+01,1.933279e+01,2.524695e+01,2.325443e+01,-1.082988,0.617026
4,average_cost_for_two,0.000000,2.500002e+07,2.500002e+07,7.152113e+03,2.900000e+02,2.905875e+05,84.578959,7271.931662
5,has_table_booking,0.000000,1.000000e+00,1.000000e+00,6.217617e-02,0.000000e+00,2.414752e-01,3.626965,11.157839
6,has_online_delivery,0.000000,1.000000e+00,1.000000e+00,3.524645e-01,0.000000e+00,4.777377e-01,0.617766,-1.618796
7,is_delivering_now,0.000000,1.000000e+00,1.000000e+00,1.743058e-01,0.000000e+00,3.793723e-01,1.717359,0.949573
8,switch_to_order_menu,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
9,price_range,1.000000,4.000000e+00,3.000000e+00,2.624552e+00,3.000000e+00,9.597301e-01,-0.040693,-0.983179


# 2.0 Data Preprocessing

In [26]:
df2 = df1.copy()

## 2.1. Remove Duplicate data

In [27]:
# If there is duplicate data with same id, it will be removed 
df2 = df2.drop_duplicates(subset=['restaurant_id'])



## 2.2. Remove NA

In [28]:
df2 = df2.dropna()

# Imprimir as dimensões dos dados
show_dataframe_dimensions(df2)

Number of Rows: 6929
Number of Columns: 21


## 2.3. Change Data Types

In [29]:
# Converter as colunas para booleanas
df2['has_table_booking'] = df2['has_table_booking'].astype(bool)
df2['has_online_delivery'] = df2['has_online_delivery'].astype(bool)
df2['is_delivering_now'] = df2['is_delivering_now'].astype(bool)

## 2.4. Create New Features

In [30]:
df2['country'] = df2['country_code'].apply(country_name)
df2['price_type'] = df2['price_range'].apply(create_price_tye)
df2['color_name'] = df2['rating_color'].apply(color_name)
df2["cuisines_"] = df2.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

In [31]:
df2.to_csv("data/processed/data.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data/processed'

# 3.0. Answering Questions

In [59]:
# df3 = pd.read_csv("data/processed/data.csv")
df3 = process_data("../data/raw/zomato.csv")

In [60]:
df3.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,country,price_type,color_name,cuisines_
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen,Italian
1,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,0,4,4.7,3F7E00,Excellent,469,Philippines,gourmet,darkgreen,European
2,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,0,3,4.4,5BA829,Very Good,867,Philippines,expensive,green,Filipino
3,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,3,4.4,5BA829,Very Good,858,Philippines,expensive,green,American
4,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,0,3,4.7,3F7E00,Excellent,930,Philippines,expensive,darkgreen,Filipino


In [61]:
df3.reset_index(drop=True).loc[1,:]

restaurant_id                                                     6314542
restaurant_name                                                 Blackbird
country_code                                                          162
city                                                          Makati City
address                 Nielson Tower, Ayala Triangle Gardens, Salcedo...
locality                Ayala Triangle Gardens, Salcedo Village, Makat...
locality_verbose        Ayala Triangle Gardens, Salcedo Village, Makat...
longitude                                                      121.024562
latitude                                                        14.556042
cuisines                                                  European, Asian
average_cost_for_two                                                 3100
currency                                                 Botswana Pula(P)
has_table_booking                                                   False
has_online_delivery                   

In [57]:
df3.reset_index(drop=True).loc[2,:]

restaurant_id                                                     6301293
restaurant_name                                                  Banapple
country_code                                                          162
city                                                          Makati City
address                 Ayala Triangle Gardens, Salcedo Village, Makat...
locality                Ayala Triangle Gardens, Salcedo Village, Makat...
locality_verbose        Ayala Triangle Gardens, Salcedo Village, Makat...
longitude                                                      121.023171
latitude                                                        14.556196
cuisines                              Filipino, American, Italian, Bakery
average_cost_for_two                                                  800
currency                                                 Botswana Pula(P)
has_table_booking                                                   False
has_online_delivery                   

## General

### 1. How many unique restaurants are registered?

In [ ]:
df3.restaurant_id.nunique()

6929

### 2. How many unique countries are registered?

In [ ]:
df3['country'].nunique()

15

### 3. How many unique cities are registered?

In [ ]:
df3['city'].nunique()

125

### 4. What is the total number of assessments carried out?

In [ ]:
df3['votes'].sum()

4194533

### 5. What are the total types of cuisine registered?

In [ ]:
df3['cuisines_'].nunique()

165

## Country

### 1. What is the name of the country that has the most registered cities?

In [ ]:
(df3[['country', 'city']].groupby('country')
                         .count()
                         .sort_values(by='city', ascending=False)
                         .reset_index().loc[0, 'country'])

'India'

### 2. What is the name of the country that has the most registered restaurants?

In [ ]:
(df3[['country', 'restaurant_id']].groupby('country')
                         .count()
                         .sort_values(by='restaurant_id', ascending=False)
                         .reset_index().loc[0, 'country'])

'India'

### 3. What is the name of the country that has the most restaurants with an average rating equal to or greater than 4 registered?

In [ ]:
# What is the name of the country that has the most restaurants with an average rating equal to or greater than 4 registered?

(df3.loc[df3['aggregate_rating'] >= 4.0,].groupby('country')['restaurant_id']
                                         .count()
                                         .sort_values(ascending=False)
                                         .reset_index().iloc[0,0])


'India'

### 4. What is the name of the country that has the most types of cuisine different?

In [ ]:
df3.groupby('country')['cuisines_'].nunique().idxmax()

'India'

### 5. What is the name of the country that has the largest number of evaluations carried out?

In [ ]:
df3.groupby('country')['votes'].sum().idxmax()

'India'

### 6. What is the name of the country that has the largest number of restaurants that make delivery?

In [ ]:
df3.groupby('country')['is_delivering_now'].sum().idxmax()

'India'

### 7. What is the name of the country that has the largest number of restaurants that accept reservations?

In [ ]:
df3.groupby('country')['has_table_booking'].sum().idxmax()

'India'

### 8. What is the name of the country that has, on average, the highest number of reviews registered?

In [ ]:
df3.groupby('country')['votes'].mean().idxmax()

'Indonesia'

### 9. What is the name of the country that has, on average, the highest recorded average score?

In [ ]:
df3.groupby('country')['aggregate_rating'].mean().idxmax()

'Indonesia'

### 10. What is the name of the country that has, on average, the lowest average score recorded?

In [ ]:
df3.groupby('country')['aggregate_rating'].mean().idxmin()

'Brazil'

### 11. What is the average price of a dish for two per country?

In [ ]:
(df3.groupby('country')['average_cost_for_two'].mean()
                                               .sort_values(ascending=False)
                                               .reset_index(name='avg_price_dish_for_2'))

,country,avg_price_dish_for_2
0,Indonesia,303000.000000
1,Australia,138959.783333
2,Sri Lanka,2579.375000
3,Philippines,1227.825000
4,India,704.400514
5,South Africa,339.228324
6,Qatar,174.000000
7,United Arab Emirates,153.716667
8,Singapure,141.437500
9,Brazil,138.812500


### 12. What are the top 10 countries with the most restaurants?

In [ ]:
# Group the data by country and count the unique restaurant IDs
top_countries = (df3.groupby('country')['restaurant_id']
                    .nunique()  # Count unique restaurant IDs
                    .sort_values(ascending=False)  # Sort in descending order
                    .reset_index(name='number_of_restaurants'))

top_countries              

,country,number_of_restaurants
0,India,3111
1,United States of America,1374
2,England,400
3,South Africa,346
4,United Arab Emirates,300
5,Brazil,240
6,New Zeland,239
7,Australia,180
8,Canada,180
9,Turkey,159


In [ ]:
# Criar o gráfico de barras
fig = px.bar(top_countries, x='country', y='number_of_restaurants',
             labels={'country': 'Country', 'number_of_restaurants': 'Number of Restaurants'},
             text='number_of_restaurants',
             height=550)

# Ajustar a exibição dos rótulos de texto
fig.update_traces(textposition='outside')

# Remove the legend
fig.update_layout(showlegend=False)

# Exibir gráfico
fig.show()

### 13. What are the 10 countries with the most registered cities

In [ ]:
# What are the 10 countries with the most registered cities
top_cities_by_country = (df3.groupby('country')['city']
                            .nunique()
                            .sort_values(ascending=False)
                            .reset_index(name='number_of_cities'))

top_cities_by_country

,country,number_of_cities
0,India,49
1,United States of America,22
2,Philippines,12
3,South Africa,12
4,England,5
5,New Zeland,4
6,United Arab Emirates,4
7,Australia,3
8,Brazil,3
9,Canada,3


In [ ]:
# Criar o gráfico de barras
fig = px.bar(top_cities_by_country, x='country', y='number_of_cities',
             labels={'country': 'Country', 'number_of_cities': 'Number of Cities'},
             text='number_of_cities',
             height=550)

# Ajustar a exibição dos rótulos de texto
fig.update_traces(textposition='outside')

# Remove the legend
fig.update_layout(showlegend=False)

# Exibir gráfico
fig.show()

### 14. What is the average rating of restaurants in each country?

In [ ]:
average_ratings= (df3.groupby('country')['aggregate_rating']
                     .mean()
                     .sort_values(ascending=False)
                     .reset_index(name='mean_aggregate_rating'))

average_ratings['mean_aggregate_rating'] = round(average_ratings['mean_aggregate_rating'], 1)

average_ratings.columns = ['Country', 'Mean Aggregate Rating']

average_ratings

,Country,Mean Aggregate Rating
0,Indonesia,4.6
1,Philippines,4.5
2,Singapure,4.4
3,United States of America,4.4
4,Australia,4.4
5,Canada,4.3
6,Turkey,4.3
7,Qatar,4.2
8,New Zeland,4.2
9,South Africa,4.1


In [ ]:
# Create a choropleth map using Plotly Express
fig = px.choropleth(average_ratings, 
                    locations='Country', 
                    locationmode='country names',
                    color='Mean Aggregate Rating',
                    hover_name='Country',
                    color_continuous_scale='Bluered_r')  # Color scale   

# Show the map
fig.show()

In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(average_ratings, x='Country', y='Mean Aggregate Rating',
             height=600,  # Adjust the height as needed
             color='Mean Aggregate Rating',  # Color based on mean aggregate rating
             color_continuous_scale='Bluered_r')  # Color palette

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 15. What is the average price rating of restaurants in each country?

In [ ]:
average_price_ratings= (df3.groupby('country')['price_range']
                     .mean()
                     .sort_values(ascending=False)
                     .reset_index(name='mean_price_range'))

average_price_ratings['mean_price_range'] = round(average_price_ratings['mean_price_range'], 1)

average_price_ratings.columns = ['Country', 'Mean price Range']

average_price_ratings

,Country,Mean price Range
0,Singapure,3.7
1,Qatar,3.6
2,South Africa,3.5
3,Brazil,3.4
4,Philippines,3.2
5,Turkey,3.2
6,United Arab Emirates,3.2
7,Indonesia,3.1
8,New Zeland,3.0
9,Sri Lanka,2.8


In [ ]:
# Create a choropleth map using Plotly Express
fig = px.choropleth(average_price_ratings, 
                    locations='Country', 
                    locationmode='country names',
                    color='Mean price Range',
                    hover_name='Country',
                    color_continuous_scale='Bluered')  # Color scale   

# Show the map
fig.show()

In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(average_price_ratings, x='Country', y='Mean price Range',
             height=600,  # Adjust the height as needed
             color='Mean price Range',  # Color based on mean aggregate rating
             color_continuous_scale='Bluered')  # Color palette

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

## City

### 1. What is the name of the city that has the most registered restaurants?

In [ ]:
df3.groupby('city')['restaurant_id'].nunique().idxmax()

'Abu Dhabi'

### 2. What is the name of the city that has the most restaurants with an average rating above 4?

In [ ]:
df3[df3['aggregate_rating'] >= 4].groupby('city')['restaurant_id'].nunique().idxmax()

'London'

### 3. What is the name of the city that has the most restaurants with an average rating below 2.5?

In [ ]:
df3[df3['aggregate_rating'] <= 2.5].groupby('city')['restaurant_id'].nunique().idxmax()

'Gangtok'

### 4. What is the name of the city that has the highest average price for a dish for two?

In [ ]:
df3.groupby('city')['average_cost_for_two'].mean().idxmax()

'Adelaide'

### 5. What is the name of the city that has the largest number of different types of cuisine?

In [ ]:
df3.groupby('city')['cuisines_'].nunique().idxmax()

'Birmingham'

### 6. What is the name of the city that has the largest number of restaurants that take reservations?

In [ ]:
df3.groupby('city')['has_table_booking'].sum().idxmax()

'Bangalore'

### 7. What is the name of the city that has the largest number of restaurants that deliver?

In [ ]:
# Filter the restaurants that are delivering now
delivering_now = df3[df3['is_delivering_now']]

# Count the number of restaurants delivering now per city
city_delivery_counts = delivering_now.groupby('city')['restaurant_id'].count()

# Find the maximum count value
max_count = city_delivery_counts.max()

# Filter the cities that have the maximum count value
cities_with_max_delivery = city_delivery_counts[city_delivery_counts == max_count].index.tolist()

print(cities_with_max_delivery)


['Amritsar', 'Vadodara']


### 8. What is the name of the city that has the largest number of restaurants that accept online orders?

In [ ]:
df3[df3['has_online_delivery']].groupby('city')['restaurant_id'].count().idxmax()

'Bhopal'

### 9. What are the top 10 cities with more registered restaurants?

In [ ]:
df3.groupby('city')['restaurant_id'].nunique().sort_values(ascending=False).reset_index(name='Count').head(10)

,city,Count
0,Abu Dhabi,80
1,Mysore,80
2,Glasgow,80
3,Goa,80
4,Guwahati,80
5,Hamilton,80
6,Houston,80
7,Indore,80
8,Jaipur,80
9,Kanpur,80


### 10. What are the 10 cities that have the highest mean aggregate restaurant rating?

In [ ]:
mean_aggregate_rating = (df3.groupby(['city', 'country'])['aggregate_rating'].mean()
                                                                .sort_values(ascending=False)
                                                                .reset_index(name='mean_aggregate_rating')
                                                                .head(10))

mean_aggregate_rating

,city,country,mean_aggregate_rating
0,Muntinlupa City,Philippines,4.900000
1,Zirakpur,India,4.700000
2,Ghaziabad,India,4.700000
3,Inner City,South Africa,4.650000
4,London,England,4.610000
5,Jakarta,Indonesia,4.607895
6,New York City,United States of America,4.606667
7,Tangerang,Indonesia,4.600000
8,Las Piñas City,Philippines,4.600000
9,Pasig City,Philippines,4.583333


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(mean_aggregate_rating, x='city', y='mean_aggregate_rating',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             labels={'city': 'City', 'mean_aggregate_rating': 'Mean Aggregate Rating', 'country': 'Country'},
             category_orders={'city': mean_aggregate_rating['city'].tolist()}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 11. What are the 10 cities that have the lowest mean aggregate restaurant rating?

In [ ]:
mean_aggregate_rating = (df3.groupby(['city', 'country'])['aggregate_rating'].mean()
                                                                .sort_values(ascending=True)
                                                                .reset_index(name='mean_aggregate_rating')
                                                                .head(10))

mean_aggregate_rating

,city,country,mean_aggregate_rating
0,Gangtok,India,1.775000
1,Ooty,India,2.486250
2,Clarens,South Africa,2.780769
3,Brasília,Brazil,2.875000
4,Shimla,India,3.155696
5,São Paulo,Brazil,3.327500
6,Fujairah,United Arab Emirates,3.348333
7,Manchester,England,3.660000
8,Rio de Janeiro,Brazil,3.762500
9,Hamilton,New Zeland,3.880000


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(mean_aggregate_rating, x='city', y='mean_aggregate_rating',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             color_continuous_scale='Bluered',
             labels={'city': 'City', 'mean_aggregate_rating': 'Mean Aggregate Rating', 'country': 'Country'},
             category_orders={'city': mean_aggregate_rating['city'].tolist()})  # Ordem das categorias)  # Color palette

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 12. What are the 10 cities with the highest mean average price rating?

In [ ]:
mean_price_range = (df3.groupby(['city', 'country'])['price_range'].mean()
                                                                .sort_values(ascending=False)
                                                                .reset_index(name='mean_price_range')
                                                                .head(10))

mean_price_range

,city,country,mean_price_range
0,Midrand,South Africa,4.000000
1,East Rand,South Africa,4.000000
2,Johannesburg South,South Africa,4.000000
3,Pasay City,Philippines,4.000000
4,Roodepoort,South Africa,4.000000
5,Johannesburg,South Africa,3.777778
6,Sandton,South Africa,3.756757
7,Clarens,South Africa,3.730769
8,Singapore,Singapure,3.662500
9,Doha,Qatar,3.587500


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(mean_price_range, x='city', y='mean_price_range',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             color_continuous_scale='Bluered',
             labels={'city': 'City', 'mean_price_range': 'Mean Price', 'country': 'Country'},
             category_orders={'city': mean_price_range['city'].tolist()})  # Ordem das categorias)  # Color palette

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 13. What are the 10 cities with the lowest mean average price rating?

In [ ]:
mean_price_range = (df3.groupby(['city', 'country'])['price_range'].mean()
                                                                .sort_values()
                                                                .reset_index(name='mean_price_range')
                                                                .head(10))

mean_price_range

,city,country,mean_price_range
0,Nasik,India,1.500000
1,Bhubaneshwar,India,1.612500
2,Ludhiana,India,1.650000
3,Gangtok,India,1.708333
4,Bhopal,India,1.725000
5,Kochi,India,1.762500
6,Ranchi,India,1.762500
7,Amritsar,India,1.800000
8,Mohali,India,1.812500
9,Ooty,India,1.825000


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(mean_price_range, x='city', y='mean_price_range',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             color_continuous_scale='Bluered',
             labels={'city': 'City', 'mean_price_range': 'Mean Price', 'country': 'Country'},
             category_orders={'city': mean_price_range['city'].tolist()}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 15. What are the top 10 cities that have the most restaurants with an average rating above 4?

In [ ]:
df3.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country', 'price_type', 'color_name', 'cuisines_'],
      dtype='object')

In [ ]:
df_aux = df3[df3['aggregate_rating'] >=4.0]


count_to10_agg_rating = (df_aux.groupby(['city', 'country'])['restaurant_id']
                               .count()
                               .sort_values(ascending=False)
                               .reset_index(name='count')
                               .head(10))

count_to10_agg_rating

,city,country,count
0,London,England,80
1,Bangalore,India,79
2,Houston,United States of America,77
3,Pune,India,77
4,İstanbul,Turkey,76
5,Kolkata,India,76
6,Jakarta,Indonesia,75
7,Auckland,New Zeland,75
8,Austin,United States of America,73
9,Chennai,India,72


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(count_to10_agg_rating, x='city', y='count',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             labels={'city': 'City', 'count': 'Count', 'country': 'Country'},
             category_orders={'city': count_to10_agg_rating['city'].tolist()}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 16. What are the top 10 cities that have the most restaurants with an average rating below 2.5?

In [ ]:
df_aux = df3[df3['aggregate_rating'] < 2.5]


count_to10_agg_rating = (df_aux.groupby(['city', 'country'])['restaurant_id']
                               .count()
                               .sort_values(ascending=False)
                               .reset_index(name='count')
                               .head(10))

count_to10_agg_rating

,city,country,count
0,Gangtok,India,33
1,Ooty,India,19
2,São Paulo,Brazil,16
3,Brasília,Brazil,15
4,Rio de Janeiro,Brazil,12
5,Manchester,England,7
6,Clarens,South Africa,4
7,Shimla,India,4
8,Edinburgh,England,2
9,Hamilton,New Zeland,2


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(count_to10_agg_rating, x='city', y='count',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             labels={'city': 'City', 'count': 'Count', 'country': 'Country'},
             category_orders={'city': count_to10_agg_rating['city'].tolist()}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 17. What are the top 10 cities that have the most restaurants with different types of cuisine?

In [ ]:
df3.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'country', 'price_type', 'color_name', 'cuisines_'],
      dtype='object')

In [ ]:
unique_cuisine_by_city = (df3.groupby(['city', 'country'])['cuisines_'].nunique()
                                                          .sort_values(ascending=False)
                                                          .reset_index(name='number_of_unique_cuisines')
                                                          .head(10))

unique_cuisine_by_city

,city,country,number_of_unique_cuisines
0,Birmingham,England,32
1,Doha,Qatar,31
2,Montreal,Canada,30
3,São Paulo,Brazil,30
4,Manchester,England,30
5,Houston,United States of America,30
6,Perth,Australia,29
7,Philadelphia,United States of America,29
8,Portland,United States of America,28
9,Calgary,Canada,28


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(unique_cuisine_by_city, x='city', y='number_of_unique_cuisines',
             height=600,  # Adjust the height as needed
             color='country',  # Color based on mean aggregate rating
             labels={'city': 'City', 'number_of_unique_cuisines': 'Number of Unique Cuisines', 'country': 'Country'},
             category_orders={'city': unique_cuisine_by_city['city'].tolist()}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

## Restaurants

### 1. What is the name of the restaurant that has the most reviews?

In [ ]:
df3.loc[df3['votes'].idxmax(), 'restaurant_name']

'Bawarchi'

### 2. What is the name of the restaurant with the highest average rating?

In [ ]:
df3.sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,1]

'Indian Grill Room'

### 3. What is the name of the restaurant that has the highest value for one dish for two people?

In [ ]:
df3.sort_values(['average_cost_for_two', 'restaurant_id'], ascending=[False, True]).iloc[0,1]

"d'Arry's Verandah Restaurant"

### 4. What is the name of the restaurant with the type of Brazilian cuisine that has the lowest average rating?

In [ ]:
(df3[df3['cuisines_'] == 'Brazilian']
 .sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0,1])

'Loca Como tu Madre'

### 5. What is the name of the Brazilian cuisine restaurant, and that it is from Brazil, that Does it have the highest average rating?

In [ ]:
(df3[(df3['cuisines_'] == 'Brazilian') & (df3['country'] == 'Brazil')] 
 .sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0,1])

'Braseiro da Gávea'

### 6. Are restaurants that accept online orders also, on average, the restaurants that have the most registered reviews?

In [ ]:
(df3.groupby('has_online_delivery')['votes']
   .mean()
   .sort_values(ascending=False)
   .reset_index(name='avg_votes'))

,has_online_delivery,avg_votes
0,True,838.821664
1,False,479.421018


### 7. Are the restaurants that take reservations also, on average, the restaurants that have the highest average price for a dish for two people?

In [ ]:
(df3.groupby('has_table_booking')['average_cost_for_two']
    .mean()
    .sort_values(ascending=False)
    .reset_index(name='avg_average_cost_for_two')) 

,has_table_booking,avg_average_cost_for_two
0,True,69998.423810
1,False,3488.596866


### 8. Do Japanese cuisine restaurants in the United States of America have a higher average serving price for two people than American steakhouses (BBQ)?

In [ ]:
filtered_lines = ((df3['country'] == 'United States of America') & 
                 ((df3['cuisines_'] == 'Japanese') | (df3['cuisines_'] == 'BBQ')))

df3[filtered_lines].groupby('cuisines_')['average_cost_for_two'].mean().sort_values(ascending=False).reset_index(name='avg_average_cost_for_two')

,cuisines_,avg_average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


## Types of Cuisine

### 1. What is the name of the Italian cuisine restaurant with the highest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Italian')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Darshan'

### 2. What is the name of the Italian cuisine restaurant with the lowest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Italian')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Avenida Paulista'

### 3. What is the name of the restaurant with American cuisine with the highest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'American')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Burger & Lobster'

### 4. What is the name of the restaurant with American cuisine with the lowest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'American')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Alston Bar & Beef'

### 5. What is the name of the restaurant with Arabic cuisine with the highest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Arabian')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Mandi@36'

### 6. What is the name of the restaurant with Arabic cuisine with the lowest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Arabian')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Raful'

### 7. What is the name of the restaurant with Japanese cuisine with the highest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Japanese')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Sushi Samba'

### 8. What is the name of the restaurant with Japanese cuisine with the lowest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Japanese')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'Banzai Sushi'

### 9. What is the name of the restaurant with homemade cuisine with the highest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Home-made')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).iloc[0, 1]

'Kanaat Lokantası'

### 10. What is the name of the restaurant with homemade cuisine with the lowest average rating?

In [ ]:
lines = (df3['cuisines_'] == 'Home-made')

df3[lines].sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[True, True]).iloc[0, 1]

'GurMekan Restaurant'

### 11. What type of cuisine has the highest average value for a dish for two people?

In [ ]:
df3.groupby('cuisines_')['average_cost_for_two'].mean().idxmax()

'Modern Australian'

### 12. What type of cuisine has the highest average score?

In [ ]:
df_aux = df3.groupby('cuisines_')['aggregate_rating'].max()

highest_rating = df_aux.sort_values(ascending=False)[0]

filtered_lines = df_aux == highest_rating

df_aux[filtered_lines].reset_index(name='highest_aggregate_rating')

,cuisines_,highest_aggregate_rating
0,American,4.9
1,Asian,4.9
2,BBQ,4.9
3,Bakery,4.9
4,Beverages,4.9
5,Brazilian,4.9
6,Breakfast,4.9
7,British,4.9
8,Burger,4.9
9,Cafe,4.9


### 13. Which type of cuisine has the most restaurants that accept online orders and deliver?

In [ ]:
filteres_lines = (df3['has_online_delivery']) & (df3['is_delivering_now'])

df3[filteres_lines].groupby('cuisines_')['restaurant_id'].count().idxmax()

'North Indian'

### 14. What are the top 10 types of cuisine with the highest average rating?

In [ ]:
df_aux = (df3.groupby('cuisines_')['aggregate_rating']
    .mean()
    .sort_values(ascending=False)
    .reset_index(name='mean_aggregate_rating').head(10))

df_aux

,cuisines_,mean_aggregate_rating
0,Others,4.90
1,Ramen,4.80
2,Egyptian,4.80
3,Ottoman,4.80
4,Sunda,4.75
5,Fresh Fish,4.75
6,Author,4.70
7,Polish,4.70
8,Burmese,4.65
9,Pan Asian,4.60


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(df_aux, x='cuisines_', y='mean_aggregate_rating',
             height=600,  # Adjust the height as needed
             labels={'cuisines_': 'Cuisines', 'mean_aggregate_rating': 'Mean Aggregate Rating'}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

### 15. What are the top 10 types of cuisine with the lowest average rating?

In [ ]:
df_aux = (df3.groupby('cuisines_')['aggregate_rating']
    .mean()
    .sort_values(ascending=True)
    .reset_index(name='mean_aggregate_rating').head(10))

df_aux

,cuisines_,mean_aggregate_rating
0,Afghan,2.050000
1,Bengali,2.133333
2,Tea,2.460000
3,Tibetan,2.800000
4,Durban,2.900000
5,Cantonese,3.400000
6,Armenian,3.400000
7,Brazilian,3.457534
8,Nepalese,3.516667
9,Iranian,3.600000


In [ ]:
# Create a bar plot using Plotly Express
fig = px.bar(df_aux, x='cuisines_', y='mean_aggregate_rating',
             height=600,  # Adjust the height as needed
             labels={'cuisines_': 'Cuisines', 'mean_aggregate_rating': 'Mean Aggregate Rating'}) 

# Adjust text position for better visibility
fig.update_traces(textposition='outside')

# Show the plot
fig.show()